In [2]:
import os
import PIL
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import pathlib

import folium
import base64
from folium import IFrame
from io import BytesIO

In [3]:
def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1] / 60.0 + coords[2] / 3600.0
    if ref == "S" or ref == "W":
        decimal_degrees = -decimal_degrees
    return decimal_degrees

In [5]:
def get_exif(file_name):
    image = PIL.Image.open(file_name)
    return image.getexif()


def get_geo(exif):
    for key, value in TAGS.items():
        if value == "GPSInfo":
            break
    if value != "GPSInfo":
        raise Exception()
    gps_info = exif.get_ifd(key)
    return {GPSTAGS.get(key, key): value for key, value in gps_info.items()}


def get_exif_old(image_file_path):
    exif_table = {}
    image = PIL.Image.open(image_file_path)
    info = image.getexif()

    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        exif_table[decoded] = value

    return exif_table

In [6]:
base_dir = pathlib.Path(r"/home/krzysztof/tmp/wtst")

In [7]:
def get_coords(img_path):
    try:
        geo = get_geo(get_exif(img_path))
        return [
            decimal_coords(geo["GPSLatitude"], geo["GPSLatitudeRef"]),
            decimal_coords(geo["GPSLongitude"], geo["GPSLongitudeRef"]),
        ]
    except Exception:
        return None

In [11]:
for img in os.listdir(base_dir):
    # if pathlib.Path(img).suffix == ".pdf":
    #     print("pdf")
    #     continue

    if pathlib.Path(img).suffix.lower() not in [".jpg", ".jpeg"]:
        print("not jpg")
        continue
    
    p = base_dir / img

    print(get_exif_old(p).get("Model"), img)
    try:
        print(get_geo(get_exif(p)))
    except Exception as e:
        print(get_exif_old(p))

iPhone SE (2nd generation) IMG_1623.JPG
{'GPSLatitudeRef': 'N', 'GPSLatitude': (40.0, 41.0, 17.93), 'GPSLongitudeRef': 'W', 'GPSLongitude': (73.0, 58.0, 42.88), 'GPSAltitudeRef': b'\x00', 'GPSAltitude': 12.13807469126618, 'GPSSpeedRef': 'K', 'GPSSpeed': 0.3422743941627916, 'GPSImgDirectionRef': 'T', 'GPSImgDirection': 78.15117647058824, 'GPSDestBearingRef': 'T', 'GPSDestBearing': 78.15117647058824, 'GPSHPositioningError': 14.396286829005785}
iPhone SE (2nd generation) IMG_1525.JPG
{'GPSLatitudeRef': 'N', 'GPSLatitude': (40.0, 42.0, 52.96), 'GPSLongitudeRef': 'W', 'GPSLongitude': (74.0, 0.0, 41.72), 'GPSAltitudeRef': b'\x00', 'GPSAltitude': 37.372932447055184, 'GPSSpeedRef': 'K', 'GPSSpeed': 0.0, 'GPSImgDirectionRef': 'T', 'GPSImgDirection': 246.760177646188, 'GPSDestBearingRef': 'T', 'GPSDestBearing': 246.760177646188, 'GPSHPositioningError': 58.0}
iPhone SE (2nd generation) IMG_1639.JPG
{'GPSLatitudeRef': 'N', 'GPSLatitude': (40.0, 42.0, 5.92), 'GPSLongitudeRef': 'W', 'GPSLongitude': 

In [ ]:
def add_marker(p, m):
    image = Image.open(p)
    image = image.resize((400, 350), Image.LANCZOS)
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    encoded = base64.b64encode(buffered.getvalue())
    # encoded = buffered
    html = '<img src="data:image/png;base64,{}">'.format
    iframe = IFrame(html(encoded.decode("UTF-8")), width=400, height=350)
    #     htmlcode = """<div>
    # <img src="file://{p}" alt="x" width="400" height="400">
    # </div>""".format(p=p)
    # iframe = IFrame(svg(encoded.decode('UTF-8'), width, height) , width=width*fat_wh, height=height*fat_wh)
    popup = folium.Popup(iframe, max_width=400)
    # iframe = IFrame(html=htmlcode, width=500, height=300)
    # popup = folium.Popup(iframe, max_width=400)
    coords = get_coords(p)
    if not coords:
        return m

    folium.Marker(
        location=get_coords(p),
        # tooltip=html,
        popup=popup,
        # icon=folium.Icon(color = 'gray')
    ).add_to(m)

    return m

In [ ]:
# [get_coords(os.path.join(base_dir, p)) for p in os.listdir(base_dir)]

In [ ]:
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)
for p in [os.path.join(base_dir, p) for p in os.listdir(base_dir)]:
    if not p:
        continue
    # folium.Marker(p).add_to(m)
    m = add_marker(p, m)

m